In [44]:
import os
os.getcwd()

'/media/hamza/Disque local'

In [58]:
%pwd

'/media/hamza/Disque local/Hamza Bouajila/Portfolio/WineQualityPrediction-MLOPS'

In [57]:
# os.chdir("WineQualityPrediction-MLOPS")

In [56]:
%pwd

'/media/hamza/Disque local/Hamza Bouajila/Portfolio/WineQualityPrediction-MLOPS'

In [59]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [60]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

In [61]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [76]:
import os
import zipfile
from src.mlProject import logger
from src.mlProject.utils.common import get_size
import requests

In [80]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            response = requests.get(self.config.source_URL, allow_redirects=True)

            # Save the downloaded file to the specified path
            with open(self.config.local_data_file   , 'wb') as file:
                file.write(response.content)
            logger.info(f"{response.headers} download! with following info")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [82]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    print(data_ingestion_config)
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-12-08 23:09:53,213: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-12-08 23:09:53,217: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-08 23:09:53,222: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-08 23:09:53,224: INFO: common: created directory at: artifacts]
[2024-12-08 23:09:53,225: INFO: common: created directory at: artifacts/data_ingestion]
DataIngestionConfig(root_dir='artifacts/data_ingestion', source_URL='https://www.kaggle.com/api/v1/datasets/download/johnsmith88/heart-disease-dataset', local_data_file='artifacts/data_ingestion/data.zip', unzip_dir='artifacts/data_ingestion')
[2024-12-08 23:09:54,746: INFO: 1822926028: {'X-GUploader-UploadID': 'AFiumC66XdwhRfrihbsS_dMH3B-jgfGwYdnGSA3i6rd93ujpXxLhTNAKwE-Aifr8mB8NdNvPuOrzyVc', 'Expires': 'Sun, 08 Dec 2024 22:09:54 GMT', 'Date': 'Sun, 08 Dec 2024 22:09:54 GMT', 'Cache-Control': 'private, max-age=0', 'Last-Modified': 'Fri, 25 Oct 2019 01:10:08 GMT', 'ETag': '"